In [2]:
import statsmodels.formula.api as smf
import pandas as pd

from utils.wrangle_model_data import wrangle


df = pd.read_csv('data/commodity_prices.csv')
df = wrangle(df)

In [3]:
pd.crosstab(df['Market'], df['Season'])

Season,Post Monsoon,Southwest Monsoon,Summer,Winter
Market,,,,
Aluva,3316,4823,3569,3466
Angamaly,3250,4521,3370,3246
Broadway market,1489,2428,2899,2643
Ernakulam,1180,1742,1064,856
Keezhampara VFPCK,759,687,692,787
Koovapadi VFPCK,1119,1296,1002,1238
Kothamangalam,3477,4897,3658,3602
Mazhuvannur VFPCK,1771,1270,1656,1777
Moovattupuzha,448,330,249,581


In [4]:
pd.crosstab(df['Market'], df['Year'])

Year,2023,2024,2025
Market,,,
Aluva,3694,6687,4793
Angamaly,3735,6080,4572
Broadway market,196,5734,3529
Ernakulam,1388,2242,1212
Keezhampara VFPCK,577,1453,895
Koovapadi VFPCK,945,2097,1613
Kothamangalam,3827,7249,4558
Mazhuvannur VFPCK,1334,3188,1952
Moovattupuzha,571,669,368


In [6]:
model = smf.mixedlm(
    "Modal_Price ~ C(Product_Type) + C(Season) + Year",
    data=df,
    groups=df["Market"],
    re_formula="~Year"
)
result = model.fit(method="lbfgs", reml=True)


/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2206: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2218: ConvergenceWarning: Gradient optimization failed, |grad| = 10605.390956
  warnings.warn(msg, ConvergenceWarning)
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated paramet

In [7]:
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                             Mixed Linear Model Regression Results
===============================================================================================================================
Model:                                    MixedLM                       Dependent Variable:                       Modal_Price  
No. Observations:                         146612                        Method:                                   REML         
No. Groups:                               13                            Scale:                                    3116287.9735 
Min. group size:                          1608                          Log-Likelihood:                           -1303653.7054
Max. group size:                          21947                         Converged:                                No           
Mean group size:                          11277.8                                                                              
-------------------------------------------------------------------------------------------------------------------------------
                                                                          Coef.     Std.Err.    z    P>|z|   [0.025    0.975]  
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                               105219.131 14983.170   7.022 0.000 75852.657 134585.605
C(Product_Type)[T.Amaranthus|Amaranthus|FAQ]                             -1033.368    88.751 -11.643 0.000 -1207.318   -859.419
C(Product_Type)[T.Amaranthus|Other|FAQ]                                   -517.607   100.328  -5.159 0.000  -714.246   -320.969
C(Product_Type)[T.Amla(Nelli Kai)|Amla|FAQ]                               1609.539   115.593  13.924 0.000  1382.982   1836.097
C(Product_Type)[T.Amphophalus|Amphophalus|FAQ]                            1522.570    93.070  16.359 0.000  1340.155   1704.984
C(Product_Type)[T.Amphophalus|Other|FAQ]                                  1355.064   114.308  11.855 0.000  1131.026   1579.103
C(Product_Type)[T.Arecanut(Betelnut/Supari)|Supari|FAQ]                  27113.966   109.755 247.042 0.000 26898.851  27329.081
C(Product_Type)[T.Ashgourd|Ashgourd|FAQ]                                 -2074.136    87.054 -23.826 0.000 -2244.759  -1903.514
C(Product_Type)[T.Ashgourd|Other|FAQ]                                    -2058.902    98.074 -20.993 0.000 -2251.124  -1866.681
C(Product_Type)[T.Banana - Green|Banana - Green|FAQ]                       329.157    87.746   3.751 0.000   157.177    501.137
C(Product_Type)[T.Banana - Green|Other|FAQ]                                429.632   100.257   4.285 0.000   233.131    626.132
C(Product_Type)[T.Banana|Nendra Bale|Large]                                636.867    96.457   6.603 0.000   447.816    825.919
C(Product_Type)[T.Banana|Palayamthodan|Large]                            -1564.184    94.932 -16.477 0.000 -1750.247  -1378.122
C(Product_Type)[T.Banana|Palayamthodan|Medium]                           -1500.274    92.143 -16.282 0.000 -1680.871  -1319.678
C(Product_Type)[T.Banana|Poovan|Large]                                     496.540    95.789   5.184 0.000   308.797    684.283
C(Product_Type)[T.Banana|Poovan|Medium]                                    510.328   111.449   4.579 0.000   291.892    728.765
C(Product_Type)[T.Banana|Robusta|Large]                                  -1589.984   101.191 -15.713 0.000 -1788.315  -1391.653
C(Product_Type)[T.Beetroot|Beetroot|FAQ]                                   274.166    89.739   3.055 0.002    98.282    450.051
C(Product_Type)[T.Beetroot|Other|FAQ]                                      495.148   114.265   4.333 0.000   271.194    719.103
C(Product_Type)[T.Bhindi(Ladies Finger)|Bhindi|FAQ]                       -405.973    88.006  -4.613 0.000  -578.461   -233.484
C(Product_Type)[T.Bhindi(Lad